# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Exported File from Part 1
input_data_file = "output_data/city Data.csv"

city_df = pd.read_csv(input_data_file).set_index("City_ID")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Vaini,-21.2000,-175.2000,71.76,94,75,12.66,TO,1666370642
1,Sampit,-2.5333,112.9500,76.01,97,100,1.01,ID,1666370643
2,Barrow,71.2906,-156.7887,26.62,93,100,28.77,US,1666370537
3,George Town,5.4112,100.3354,80.53,95,40,8.05,MY,1666369822
4,Busselton,-33.6500,115.3333,54.09,79,69,7.78,AU,1666370645
...,...,...,...,...,...,...,...,...,...
568,Puerto Colombia,10.9878,-74.9547,89.69,70,55,1.34,CO,1666370940
569,Verkhniy Fiagdon,42.8333,44.3053,38.07,92,93,1.57,RU,1666370940
570,Synya,65.3718,58.0387,32.61,98,100,9.95,RU,1666370941


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
gmaps.configure(api_key=g_key)
locations = city_df[['Lat','Lng']]
weights = city_df['Humidity']
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations,weights = weights)
# heatmap.max_intensity = 100
# heatmap.point_radius = 2
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
ft_weather_df = city_df.loc[(city_df["Max Temp"]<80) & (city_df["Max Temp"]>70) & (city_df["Wind Speed"]<10) & (city_df["Cloudiness"] == 0)]
ft_weather_df = ft_weather_df.dropna()
ft_weather_df = ft_weather_df.reset_index()
ft_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,9,Hilo,19.7297,-155.0900,79.29,80,0,4.61,US,1666370647
1,124,‘Āhuimanu,21.4447,-157.8378,78.51,81,0,5.75,US,1666370702
2,127,Dzaoudzi,-12.7887,45.2699,76.89,73,0,3.44,YT,1666370703
3,141,Sarai Naurang,32.8259,70.7814,76.66,22,0,2.26,PK,1666370711
4,142,Ukiah,39.1502,-123.2078,78.04,62,0,0.00,US,1666370712
5,202,Karauli,26.5000,77.0167,75.78,44,0,8.08,IN,1666370747
6,215,Mount Isa,-20.7333,139.5000,76.51,30,0,8.39,AU,1666370753
7,236,Myitkyina,25.3833,97.4000,72.45,79,0,1.70,MM,1666370764
8,247,Rāmgarh,23.6333,85.5167,75.25,76,0,4.47,IN,1666370772
9,306,Ramos Arizpe,25.5500,-100.9667,72.95,45,0,0.00,MX,1666370803


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = pd.DataFrame(ft_weather_df, columns=["City","Country","Lat","Lng"])
# hotel_df = hotel_df.reset_index()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Hilo,US,19.7297,-155.0900,
1,‘Āhuimanu,US,21.4447,-157.8378,
2,Dzaoudzi,YT,-12.7887,45.2699,
3,Sarai Naurang,PK,32.8259,70.7814,
4,Ukiah,US,39.1502,-123.2078,
5,Karauli,IN,26.5000,77.0167,
6,Mount Isa,AU,-20.7333,139.5000,
7,Myitkyina,MM,25.3833,97.4000,
8,Rāmgarh,IN,23.6333,85.5167,
9,Ramos Arizpe,MX,25.5500,-100.9667,


In [25]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/json
#   ?keyword=cruise
#   &location=-33.8670522%2C151.1957362
#   &radius=1500
#   &type=restaurant
#   &key=YOUR_API_KEY

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
response = response.json()
response

{'html_attributions': [],
 'next_page_token': 'AcYSjRhxdLdcuUrC2tB3Qe9j1tryshCKf8XQczHBuegKmjX0QWc1OIY0bg-CyZ6QErtqrexDqrU0Bn0_a3LfVSKEyOXdhldVgtWHN1wPbXneovQfbzALHdYIl-3njTosF5utylTMUrrtAvYBjaBL3B1UR0c1MbjKGgBtuQAkFS1sPwo-dJClTMu-sjRQEKtzgQMaMV5It4LQJ1UFYNLfj5BYOAb9aJ0eKyxARZaB1chtNg3Y9bbAy5Ub3Bexu_wiXIZMFIL3qhQVTe4mCEy1bIJXfigB-1as4lF6CfADk43C1ITUnTEKw-yvGB0rVejdJKMuVLUGE3uxUp5qTM03eAHMtog347KcgEUPD7qdXeDbO6mpr7xxBiQYTHK5guhxmyYjU_gZbCvgKU9lnezKp5IInIYFsmjZZay9yGN9kPY5Jz1vi1At9oUUz7yrQ7iWSMo',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 19.7283446, 'lng': -155.0664264},
    'viewport': {'northeast': {'lat': 19.72944692989272,
      'lng': -155.0650755701072},
     'southwest': {'lat': 19.72674727010728, 'lng': -155.0677752298927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pi

In [26]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
        

In [32]:
hotel_df["Hotel Name"] = hotel_name
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Hilo,US,19.7297,-155.0900,Hilo Hawaiian Hotel
1,‘Āhuimanu,US,21.4447,-157.8378,Paradise Bay Resort
2,Dzaoudzi,YT,-12.7887,45.2699,Hotel Maharajah Mayotte
3,Sarai Naurang,PK,32.8259,70.7814,NaN
4,Ukiah,US,39.1502,-123.2078,"Holiday Inn Express & Suites Ukiah, an IHG Hotel"
5,Karauli,IN,26.5000,77.0167,Hotel Prakash
6,Mount Isa,AU,-20.7333,139.5000,Isa Hotel
7,Myitkyina,MM,25.3833,97.4000,Cartel Hotel
8,Rāmgarh,IN,23.6333,85.5167,HOTEL TREAT RESIDENCY
9,Ramos Arizpe,MX,25.5500,-100.9667,one Saltillo Aeropuerto


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>‘Āhuimanu</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Dzaoudzi</dd>\n<dt>Country</dt><dd>YT</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Sarai Naurang</dd>\n<dt>Country</dt><dd>PK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Ukiah</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Karauli</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Mount Isa</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Myitkyina</dd>\n<dt>Country</dt><dd>MM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Rāmgarh</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Ramos